
#Dùng LLM có sẳn của Hugging face

-Model có chức năng tổng hợp câu (tóm tắt văn bản)
-Trả lời câu hỏi
-Đoán từ tiếp theo

In [ ]:
#import locale
locale.getpreferredencodding = lambda: "UTF-8"

#bị lỗi khi cài đặt bằng lệnh pip

In [ ]:
!pip -q install transformers torch accelerate #cài transformers và bộ tăng tốc cpu

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig

# Check nếu có GPU thì dùng, không có thì fallback về CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# Config
config = AutoConfig.from_pretrained("google/flan-t5-base", trust_remote_code=True)
config.temperature = 0.1
config.max_length = 300
config.eos_token_id = tokenizer.eos_token_id
config.pad_token_id = tokenizer.pad_token_id
config.do_sample = True

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-base",
    config=config
).to(device)


Using device: cpu


In [ ]:
input_text = "summary this text: ChatGPT is an artificial intelligence (AI) chatbot that uses natural language processing to create humanlike conversational dialogue. The language model"

input_ids = tokenizer(input_text,return_tensors="pt").input_ids.to("cpu")

#Lấy output từ model

outputs = model.generate(input_ids)

print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1737: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<pad> The chatbot uses natural language processing to create human-like dialogue.</s>


In [ ]:
input_text = "Answer the following question: Can you tell me who is the president of US?"
input_ids = tokenizer(input_text,return_tensors="pt").input_ids.to("cpu")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> Donald Trump</s>


In [ ]:
input_text = "I am hungry, i want to eat some"
input_ids = tokenizer(input_text,return_tensors="pt").input_ids.to("cpu")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> ice cream</s>


In [ ]:
#Đã chặn API rồi

import requests

API_TOKEN =""
API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-base"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    print("Status Code:", response.status_code)
    print("Response Text:", response.text)  # Xem API trả về gì
    try:
        return response.json()
    except Exception as e:
        print("Error decoding JSON:", e)
        return None

output = query({
    "inputs": "The answer to the universe is",
})

print("Output:", output)


Status Code: 404
Response Text: Not Found
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Output: None


#Đối với OpenAPI


In [ ]:
!pip install openai --upgrade
!pip install typing-extensions --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0


In [ ]:
from typing import Literal
from openai import OpenAI

In [ ]:
client = OpenAI(
    api_key =""
)

#trả phí